# 地理信息系统设计与开发（空间查询和创建缓冲区）

In [23]:
import os
import ogr
import osr
import json
import requests
import matplotlib.pyplot as plt
%matplotlib inline

## 设置输入、输出文件和缓冲区距离（与空间参考一致）

In [24]:
inputfn = 'test3.shp'
outputBufferfn = 'testBuffer4.shp'
bufferDist = 5

## 获取输入数据

In [25]:
inputds = ogr.Open(inputfn)
inputlyr = inputds.GetLayer()

## 由json创建一个包含全中国区域的multipolygon，用于空间查询

In [27]:
def createMultiPolygonFromMultiPolygonJson():
    myMultipolygon = ogr.Geometry(ogr.wkbMultiPolygon)
    ds=ogr.Open('china.json')
    lyr=ds.GetLayer() 
    for feature in lyr:
        geom=feature.geometry()
        if(geom.GetGeometryName() == 'MULTIPOLYGON'):
            for i in range(geom.GetGeometryCount()):
                ring=geom.GetGeometryRef(i)
                #print('ring:',ring)
                myMultipolygon.AddGeometry(ring)
        else:
            myMultipolygon.AddGeometry(geom)
    return myMultipolygon 

## 进行空间查询并返回查询结果

In [26]:
def spatialFilter(inputlyr):
    myMultiPolygon=createMultiPolygonFromMultiPolygonJson()
    print(myMultiPolygon.GetGeometryCount())
    print("before filter:",inputlyr.GetFeatureCount())
    inputlyr.SetSpatialFilter(myMultiPolygon)
    print("after filter:",inputlyr.GetFeatureCount())

    return inputlyr

In [28]:
result=spatialFilter(inputlyr)

288
before filter: 14011
after filter: 16


## 显示查询结果

In [32]:
resultFeat = result.GetNextFeature()
while resultFeat :
   print(resultFeat.GetField('place')) 
   resultFeat = result.GetNextFeature()

127km WSW of Aksu, China
127km WSW of Aksu, China
66km ENE of Aksu, China
59km E of Sary-Tash, Kyrgyzstan
154km ENE of Luring, China
61km NNW of Pubu, China
87km ESE of Arzak, China
31km E of Junlian, China
6km SW of Changning, China
99km E of Arzak, China
22km NNW of Biruxong, China
54km N of Ziro, India
111km SSE of Kuqa, China
3km NW of Fuji, China
84km SSE of Kuqa, China
229km SE of Kuqa, China


## 创建输出文件

In [33]:
shpdriver = ogr.GetDriverByName('ESRI Shapefile')
if os.path.exists(outputBufferfn):
    shpdriver.DeleteDataSource(outputBufferfn)
outputBufferds = shpdriver.CreateDataSource(outputBufferfn)
targetSR = osr.SpatialReference()
targetSR.ImportFromEPSG(4326) #Geo WGS84
bufferlyr = outputBufferds.CreateLayer(outputBufferfn,targetSR,geom_type=ogr.wkbPolygon)
featureDefn = bufferlyr.GetLayerDefn()

## 对查询结果建立缓冲区并输出为文件

In [34]:
for feature in inputlyr:
    ingeom = feature.GetGeometryRef()
    geomBuffer = ingeom.Buffer(bufferDist)

    outFeature = ogr.Feature(featureDefn)
    outFeature.SetGeometry(geomBuffer)
    bufferlyr.CreateFeature(outFeature)
    outFeature = None

In [35]:
outputBufferds=None